In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

## **1.1 Load Data into a Dataframe**

- upload the `wine_quality` dataset from a `.csv` using `pandas`.
  - `read_csv()` will read comma-separated values file into DataFrame.
  - `head()`will return the first $n$ rows of the dataset.


In [ ]:
wine_ds = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', delimiter=';')
wine_ds.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5


In [ ]:
wine_ds.info()

- What keys are there?
- What should be the input and what should be the target?

## **1.2 Create a Tensorflow Dataset and a Dataset Pipeline**

- split the dataset into a train, test and validation split.

In [ ]:
train_ds = wine_ds.sample(frac=.7, random_state=100) # now has 70% of the dataset, or 1119 indices 
remaining_ds = wine_ds[~wine_ds.index.isin(train_ds.index)] # whatever that is not part of 70%; another 30%
test_ds = remaining_ds.sample(frac=.5, random_state=100)
val_ds = remaining_ds[~remaining_ds.index.isin(test_ds.index)]

- separate the labels from the input and store them accordingly.

In [ ]:
target = wine_ds.take([-1], axis=1)
target[0:5]

,quality
0,5
1,5
2,5
3,6
4,5


In [ ]:
targets = [int(i) for i in target.values]
targets[0:5]

[5, 5, 5, 6, 5]

- out of the resulting dataframes, build a Tensorflow Dataset.

In [ ]:
train_tfds = tf.data.Dataset.from_tensor_slices(train_ds)
test_tfds = tf.data.Dataset.from_tensor_slices(test_ds)
val_tfds = tf.data.Dataset.from_tensor_slices(val_ds)

In [ ]:
train_tfds

<TensorSliceDataset shapes: (12,), types: tf.float64>

In [ ]:
test_tfds

<TensorSliceDataset shapes: (12,), types: tf.float64>

In [ ]:
val_tfds

<TensorSliceDataset shapes: (12,), types: tf.float64>

- write a function that receives a target and returns a target fit for a binary classification task.

- this function should compare the rank to some threshold and return a 0 for bad and a 1 for good
wine

In [ ]:
def make_binary(targets):
  threshold = wine_ds['quality'].median()
  for i, val in enumerate(targets):
    if val >= threshold:
      return 1
    elif val < threshold:
      return 0

In [ ]:
make_binary(targets)

0